In [4]:
import contextvars
import asyncio

var = contextvars.ContextVar('var', default='default')

async def print_var():
    print(f"In print_var, var = {var.get()}")

async def main():
    var.set('main value')

    print(f"In main: var = {var.get()}")

    # 异步任务会继承调用者的上下文
    task = asyncio.create_task(print_var())
    await task

    # new_context为当前context的拷贝
    new_context = contextvars.copy_context()
    new_context.run(lambda: print(f"In new context, var = {var.get()}"))

    # 创建一个全新的context
    empty_context = contextvars.Context()
    empty_context.run(lambda: print(f"In empty context, var = {var.get()}"))

await main()

In main: var = main value
In print_var, var = main value
In new context, var = main value
In empty context, var = default


In [13]:
import asyncio
import contextvars

request_id = contextvars.ContextVar('request_id')

async def handle_request(id):
    request_id.set(id)

    print(f"Start request {request_id.get()}")

    await asyncio.sleep( 1 + id * 2)

    async def nested():
        print(f"Nested in request {request_id.get()}")

    await nested()

    print(f"End request {request_id.get()}")


async def main():
    tasks = [handle_request(idx) for idx in range(3)]
    await asyncio.gather(*tasks)

await main()

Start request 0
Start request 1
Start request 2
Nested in request 0
End request 0
Nested in request 1
End request 1
Nested in request 2
End request 2


In [21]:
# 我们需要意识到上下文的一致性是由用户手动保持的，而不是由系统自动保持的
import asyncio
import contextvars

var = contextvars.ContextVar('var', default={})

async def sub2():
    print(f'in sub2, {var.get()=}')
    var.set('sub1 set')

async def sub1():
    print(f'in sub1, {var.get()=}')
    var.set('sub1 set')
    await sub2()

async def main():
    var.set('main set')
    await sub1()
    print(f'in main, {var.get()=}')

await main()

in sub1, var.get()='main set'
in sub2, var.get()='sub1 set'
in main, var.get()='sub1 set'


In [22]:
import asyncio
import contextvars

var = contextvars.ContextVar('var', default={})

# 当执行从一个内部任务（inner Task）中退出时，内部任务的上下文（context）也会结束，并且外部任务（outer Task）的上下文会被恢复。这意味着任务之间的上下文不会保持一致。
# 原因在于，当创建一个任务时，该任务的上下文是一个从当前上下文复制出来的新上下文对象。对这个新上下文对象进行的任何写操作都不会影响到外部的上下文对象。
async def coro_func(level):
    var.set(level)
    print(f"level = {level}, var.get() = {var.get()}")
    if level:
        await asyncio.create_task(coro_func(level - 1))
    print(f"level = {level}, var.get() = {var.get()}")

await coro_func(2)

level = 2, var.get() = 2
level = 1, var.get() = 1
level = 0, var.get() = 0
level = 0, var.get() = 0
level = 1, var.get() = 1
level = 2, var.get() = 2


In [23]:
import asyncio
import contextvars

var = contextvars.ContextVar('var', default={})

async def coro_func(level):
    var.set(level)
    print(f"level = {level}, var.get() = {var.get()}")
    if level:
        # context = contextvars.copy_context() if asyncio.current_task() else None

        # 3.11+的create_task()才支持context参数
        # await asyncio.create_task(coro_func(level - 1), context=context)
        async def wrapper():
            var.set(level - 1)
            await coro_func(level - 1)
        await wrapper()

    print(f"level = {level}, var.get() = {var.get()}")

await coro_func(2)

level = 2, var.get() = 2
level = 1, var.get() = 1
level = 0, var.get() = 0
level = 0, var.get() = 0
level = 1, var.get() = 0
level = 2, var.get() = 0


In [25]:
import contextvars

var = contextvars.ContextVar('var')
var.set('spam')
print(var.get())  # 'spam'

# ctx是当前上下文的一份拷贝
ctx = contextvars.copy_context()

def main():
    # 'var' was set to 'spam' before
    # calling 'copy_context()' and 'ctx.run(main)', so:
    print(var.get())  # 'spam'
    print(ctx[var])  # 'spam'

    var.set('ham')

    # Now, after setting 'var' to 'ham':
    print(var.get())  # 'ham'
    print(ctx[var])  # 'ham'

# Any changes that the 'main' function makes to 'var'
# will be contained in 'ctx'.
ctx.run(main)

# The 'main()' function was run in the 'ctx' context,
# so changes to 'var' are contained in it:
print(ctx[var])  # 'ham'

# However, outside of 'ctx', 'var' is still set to 'spam':
print(var.get())  # 'spam'

spam
spam
spam
ham
ham
ham
spam
